In [97]:
import itertools

In [123]:
V = list(itertools.combinations(range(1, 12), 4)) # list of vertices, each vertex being a tuple of four numbers such as (1, 2, 3, 4)
G = {v:{} for v in V} # The Kneser graph as an adjacency matrix implemented with dictionaries: G[u][v] is True if and only if u <-> v is an edge.
disjoint_table = {v:set() for v in V} # For each vertex v, returns the neighborhood of v as a set.
intersect_table = {v:set() for v in V} # For each vertex v, returns all vertices with no edge with v.

for i in range(len(V)):
    for j in range(i+1, len(V)):
        v = V[i]
        w = V[j]
        if set(v).isdisjoint(set(w)):
            G[v][w] = True
            G[w][v] = True
            disjoint_table[v].add(w)
            disjoint_table[w].add(v)
        else:
            G[v][w] = False
            G[w][v] = False
            intersect_table[v].add(w)
            intersect_table[w].add(v)

In [130]:
print("|V(G)| = " + str(len(V)))
tot = 0
for v, l in disjoint_table.items():
    tot += len(l)
print("|E(G)| = " + str(tot // 2))
print("|d(G)| = " + str(len(disjoint_table[(1,2,3,4)])))

|V(G)| = 330
|E(G)| = 5775
|d(G)| = 35


In [124]:
def add_edge(g):
    """
    Adds another edge to g that is disjoint from everything in g.
    Returns set of all possible edge additions.
    g must be a tuple of vertices in G.
    g represents the graph of edges/singletons we're building up.
    """
    result = set()
    a = intersect_table[g[0]]
    for i in range(1, len(g)):
        a &= intersect_table[g[i]]
    for v in a:
        for w in a:
            if v != w and G[v][w]:
                result.add(tuple(list(g) + [v, w]))
    return result

In [125]:
for (u, v, w, x) in add_edge(((1,2,3,4), (5,6,7,8))):
    assert G[u][v] and G[w][x] and not G[u][w] and not G[u][x] and not G[v][w] and not G[v][x]

In [126]:
edge_one = {((1,2,3,4), (5,6,7,8))} # possibilities with one edge
edge_two = set() # possibilities with two edges
# We can assume WLOG that one of the vertices of the second edge contains 1,5.
for h in add_edge(((1,2,3,4), (5,6,7,8))):
    if 1 in h[2] and 5 in h[2]:
        edge_two.add(h)

print(len(edge_two))
tot = 0
for g in edge_two:
    if g[2] == (1, 2, 3, 5):
        tot += 1
print(tot)

960
19


In [127]:
edge_three = set()
for g in edge_two:
    for h in add_edge(g):
        edge_three.add(h)

print(len(edge_three))

14582


In [128]:
def min_cutset_size(g):
    """
    Returns "minimum size cutset" must be if Kneser graph is disconnected to g.
    Calculates the union of all neighborhoods of vertices in g.
    g must be a tuple of vertices in G.
    """
    cutset = disjoint_table[g[0]]
    for i in range(1, len(g)):
        cutset = cutset.union(disjoint_table[g[i]])
    for v in g:
        if v in cutset:
            cutset.remove(v)
    return len(cutset)

In [129]:
min_c = 330
for g in edge_three:
    min_c = min(min_c, min_cutset_size(g))
print(min_c)

131
